# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

In [ ]:
!pip install pyconll
!pip install spacy_udpipe

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

import random
import pandas as pd
import numpy as np
import re
import collections
import copy
import datetime
import traceback

# import dlnlputils
# from dlnlputils.data import tokenize_corpus, build_vocabulary, \
#     character_tokenize, pos_corpus_to_tensor, POSTagger
# from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed


def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [2]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
!mkdir datasets
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

mkdir: невозможно создать каталог «datasets»: Файл существует
--2022-05-18 00:02:32--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 40736565 (39M) [text/plain]
Сохранение в: «./datasets/ru_syntagrus-ud-train.conllu»

./datasets/ru_synta 100%[===================>]  38,85M  6,34MB/s    за 8,6s    

2022-05-18 00:02:41 (4,51 MB/s) - «./datasets/ru_syntagrus-ud-train.conllu» сохранён [40736565/40736565]

--2022-05-18 00:02:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.110.

In [2]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
full_train = pyconll.load_from_file('./datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./datasets/ru_syntagrus-ud-dev.conllu')

In [3]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [12]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [13]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [14]:
import re
import collections

TOKEN_RE = re.compile(r'[\w\d]+')

def tokenize_text_simple_regex(txt, min_token_size=4):
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]


def character_tokenize(txt):
    return list(txt)


def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
    return [tokenizer(text, **tokenizer_kwargs) for text in texts]

def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq


train_char_tokenized = tokenize_corpus(all_train_texts, 
                                       tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, 
                                             max_doc_freq=1.0, 
                                             min_count=5, 
                                             pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 142
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


Т.к. часть речи зависит от структуры слова - будем использовать отдельные символьны.

In [15]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [16]:
full_train

In [17]:
def pos_corpus_to_tensor(sentences, char2id, label2id, max_sent_len, max_token_len):
    inputs = torch.zeros((len(sentences), max_sent_len, max_token_len + 2), dtype=torch.long) # +2 для обозначения начала и конца слова
    targets = torch.zeros((len(sentences), max_sent_len), dtype=torch.long)

    for sent_i, sent in enumerate(sentences):
        for token_i, token in enumerate(sent):
            targets[sent_i, token_i] = label2id.get(token.upos, 0)
            if token.form is None:
                continue
            for char_i, char in enumerate(token.form):
                inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0) # с учётом стартового нуля для обозначения начала токена

    return inputs, targets


train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, 
                                                  MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels) # принимает на вход список тензоров - символы с метками

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, 
                                                MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

Первые пять слов второго предложения

In [18]:
train_inputs[1][:5]

tensor([[ 0, 38,  4, 25,  4, 11, 19,  7,  6, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2, 23, 11,  4,  9,  5,  7,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 17, 16, 10,  4, 12, 11,  3,  7,  6, 20,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  9, 12, 20, 21,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 15,  3,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [19]:
train_labels[1][:5]

tensor([ 8,  1,  8,  8, 12])

## Вспомогательная свёрточная архитектура

In [20]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            conv_block = nn.Sequential(
                conv_layer(
                    features_num, 
                    features_num, 
                    kernel_size, 
                    padding=kernel_size//2 # не меняет размер выхода
                ),
                nn.Dropout(dropout),
                nn.LeakyReLU()
            )
            layers.append(conv_block)
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers: # ResNet структура
            x = x + layer(x) 
        return x

## Предсказание частей речи на уровне отдельных токенов

Без учёта информации из соседних токенов.
Идея - рассмотреть с помощью SkipConnection разные n-граммы символов слова.

In [12]:
class SingleTokenPOSTagger(nn.Module): 
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        #выделение каждого токена в отдельный батч
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len) # (BatchSize*MaxSentenceLen) x MaxTokenLen
        
        # получение эмбеддингов символов
        char_embeddings = self.char_embeddings(tokens_flat)  # (BatchSize*MaxSentenceLen) x MaxTokenLen x EmbSize
        # смена размерности для подачи в сеть - размер эмбеддинга является аналогом числа каналов в изображении
        char_embeddings = char_embeddings.permute(0, 2, 1)  # (BatchSize*MaxSentenceLen) x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        # получение пулинга от всех символов токена
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        
        # обратное разделение предложений
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [13]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), 
                                          embedding_size=64, layers_n=3, 
                                          kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 47314


In [14]:
def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))


def print_grad_stats(model):
    mean = 0
    std = 0
    norm = 1e-5
    for param in model.parameters():
        grad = getattr(param, 'grad', None)
        if grad is not None:
            mean += grad.data.abs().mean()
            std += grad.data.std()
            norm += 1
    mean /= norm
    std /= norm
    print(f'Mean grad {mean}, std {std}, n {norm}')


def train_eval_loop(model, 
                    train_dataset, 
                    val_dataset, 
                    criterion,
                    lr=1e-4, 
                    epoch_n=10, 
                    batch_size=32,
                    device=None, 
                    early_stopping_patience=10, 
                    l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")
    
    # перенос модели на устройство вычисления
    device = torch.device(device)
    model.to(device)

    # создение оптимизатора
    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), 
                                     lr=lr, 
                                     weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), 
                                   lr=lr)

    # настройка изменения скорости обучения
    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    # создаём dataloader - класс pytorch, позволяющий в многопоточном режиме собирать датасеты
    train_dataloader = data_loader_ctor(train_dataset, 
                                        batch_size=batch_size, 
                                        shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    
    val_dataloader = data_loader_ctor(val_dataset, 
                                      batch_size=batch_size, 
                                      shuffle=False,
                                      num_workers=dataloader_workers_n)
    
    # инициализация сохранения лучших параметров 
    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    # цикл обучения
    for epoch_i in range(epoch_n):
        try:
            epoch_start = datetime.datetime.now()
            print('Эпоха {}'.format(epoch_i))

            model.train() # перевод модели в режим обучения, необх. для корр. dropout и batchnorm

            mean_train_loss = 0
            train_batches_n = 0
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                # указанное количество шагов в эпоху
                if batch_i > max_batches_per_epoch_train:
                    break

                # копирование данных на устройство модели
                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                pred = model(batch_x) # прямой проход
                loss = criterion(pred, batch_y) # критерий ошибки

                model.zero_grad() # очистка градиентов с прошлого шага
                loss.backward() # находим новые градиенты
                optimizer.step() # делаем градиентный шаг

                mean_train_loss += float(loss)
                train_batches_n += 1

            mean_train_loss /= train_batches_n
            print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n,
                                                           (datetime.datetime.now() - epoch_start).total_seconds()))
            print('Среднее значение функции потерь на обучении', mean_train_loss)

            
            ### Валидация

            model.eval() # перевод модели в режим предсказания
            mean_val_loss = 0
            val_batches_n = 0

            with torch.no_grad(): # выключение сохранения градиентов - экономит ресурсы
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        break

                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)

                    mean_val_loss += float(loss)
                    val_batches_n += 1

            mean_val_loss /= val_batches_n
            print('Среднее значение функции потерь на валидации', mean_val_loss)

            # сохранение лучшей модели
            if mean_val_loss < best_val_loss:
                best_epoch_i = epoch_i
                best_val_loss = mean_val_loss
                best_model = copy.deepcopy(model)
                print('Новая лучшая модель!')
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            # если задано расписание - обновить lr
            if lr_scheduler is not None:
                lr_scheduler.step(mean_val_loss)

            print()
        
        # обработка исключений
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
            
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break

    return best_val_loss, best_model

(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Device: cuda
Эпоха 0
Эпоха: 384 итераций, 12.50 сек
Среднее значение функции потерь на обучении 0.09543518974775604
Среднее значение функции потерь на валидации 0.03880157876796651
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 12.47 сек
Среднее значение функции потерь на обучении 0.030673151505955804
Среднее значение функции потерь на валидации 0.030709311015682646
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 12.88 сек
Среднее значение функции потерь на обучении 0.02640804090333404
Среднее значение функции потерь на валидации 0.027459877533930362
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 12.43 сек
Среднее значение функции потерь на обучении 0.024159870682827506
Среднее значение функции потерь на валидации 0.02633651292486356
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 12.00 сек
Среднее значение функции потерь на обучении 0.022824100757134147
Среднее значение функции потерь на валидации 0.026661730186995303

Эпоха 5
Эпоха: 384 итераций, 11.91 сек
Среднее значе

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_single_token_model.state_dict(), './models/single_token_pos.pth')

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
single_token_model.load_state_dict(torch.load('./models/single_token_pos.pth'))

In [59]:
import tqdm

def predict_with_model(model, 
                       dataset, 
                       device=None, 
                       batch_size=32, 
                       num_workers=0, 
                       return_labels=False):
    """
    :param model: torch.nn.Module - обученная модель
    :param dataset: torch.utils.data.Dataset - данные для применения модели
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :return: numpy.array размерности len(dataset) x *
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    device = torch.device(device)
    model.to(device)
    model.eval()

    dataloader = DataLoader(dataset, 
                            batch_size=batch_size, 
                            shuffle=False, 
                            num_workers=num_workers)
    
    results_by_batch = []
    labels = []
    with torch.no_grad():
        for batch_x, batch_y in tqdm.tqdm(dataloader, 
                                          total=len(dataset)/batch_size):
            batch_x = copy_data_to_device(batch_x, device)

            if return_labels:
                labels.append(batch_y.numpy())

            batch_pred = model(batch_x)
            results_by_batch.append(batch_pred.detach().cpu().numpy())

    if return_labels:
        return (np.concatenate(results_by_batch, axis=0), 
                np.concatenate(labels, axis=0))
    else:
        return np.concatenate(results_by_batch, axis=0)

train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:04, 157.38it/s]                             


Среднее значение функции потерь на обучении 0.019462604075670242
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.96      0.88      0.92     42552
         ADP       1.00      0.99      0.99     39344
         ADV       0.79      0.95      0.87     22448
         AUX       0.86      0.72      0.79      3531
       CCONJ       0.88      0.97      0.93     15168
         DET       0.76      0.92      0.83     10780
        INTJ       0.00      0.00      0.00        50
        NOUN       0.98      0.94      0.96    103538
         NUM       0.95      0.92      0.93      6764
        PART       0.97      0.78      0.86     13540
        PRON       0.95      0.77      0.85     18732
       PROPN       0.76      0.97      0.85     14858
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.80      0.73      0.76      8022
         SYM       1.00      0.99      0.99       420
        VERB    

279it [00:01, 159.41it/s]                              


Среднее значение функции потерь на валидации 0.02406851202249527
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.94      0.85      0.89     14793
         ADP       1.00      0.99      0.99     13717
         ADV       0.76      0.94      0.84      7714
         AUX       0.88      0.68      0.77      1391
       CCONJ       0.90      0.97      0.93      5672
         DET       0.76      0.87      0.81      4265
        INTJ       0.00      0.00      0.00        24
        NOUN       0.97      0.92      0.94     36238
         NUM       0.90      0.90      0.90      2119
        PART       0.96      0.76      0.85      5117
        PRON       0.94      0.78      0.85      7444
       PROPN       0.74      0.96      0.83      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.77      0.68      0.72      2859
         SYM       1.00      0.85      0.92        62
        VERB    

## Предсказание частей речи на уровне предложений (с учётом контекста)

Позволяет учитывать частиречную омонимию.
Идея - на первом этапе проанализировать структуру слова, а потом учесть контекст

In [66]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, 
                 single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        # первая часть - совпадает, получаем вектора токенов
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        # вторая часть - работаем на уровне токенов для учёта контекста
        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [67]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84370


In [68]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=5e-3,
                                              epoch_n=10,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=100,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                         factor=0.5,
                                                                                                                         verbose=True))

Device: cuda
Эпоха 0
Эпоха: 384 итераций, 12.58 сек
Среднее значение функции потерь на обучении 0.08636830801939747
Среднее значение функции потерь на валидации 0.030445895166975438
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 12.70 сек
Среднее значение функции потерь на обучении 0.027479111403711915
Среднее значение функции потерь на валидации 0.021583563989341848
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 12.60 сек
Среднее значение функции потерь на обучении 0.022410835658471722
Среднее значение функции потерь на валидации 0.02046697088839984
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 12.57 сек
Среднее значение функции потерь на обучении 0.020179432998702396
Среднее значение функции потерь на валидации 0.017994869943007384
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 12.81 сек
Среднее значение функции потерь на обучении 0.018597769439414453
Среднее значение функции потерь на валидации 0.015888662160475655
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_sentence_level_model.state_dict(), './models/sentence_level_pos.pth')

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
sentence_level_model.load_state_dict(torch.load('./models/sentence_level_pos.pth'))

In [70]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:04, 162.99it/s]                             


Среднее значение функции потерь на обучении 0.010993163101375103
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.93      0.94      0.93     42552
         ADP       1.00      0.99      0.99     39344
         ADV       0.92      0.88      0.90     22448
         AUX       0.92      0.74      0.82      3531
       CCONJ       0.94      0.98      0.96     15168
         DET       0.94      0.88      0.91     10780
        INTJ       0.83      0.10      0.18        50
        NOUN       0.97      0.97      0.97    103538
         NUM       0.91      0.96      0.93      6764
        PART       0.97      0.88      0.92     13540
        PRON       0.93      0.92      0.93     18732
       PROPN       0.91      0.98      0.94     14858
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.85      0.95      0.90      8022
         SYM       0.98      1.00      0.99       420
        VERB    

279it [00:01, 159.98it/s]                              


Среднее значение функции потерь на валидации 0.0157766230404377
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1574439
         ADJ       0.90      0.92      0.91     14793
         ADP       0.99      0.99      0.99     13717
         ADV       0.89      0.86      0.88      7714
         AUX       0.90      0.73      0.81      1391
       CCONJ       0.95      0.97      0.96      5672
         DET       0.94      0.82      0.88      4265
        INTJ       0.00      0.00      0.00        24
        NOUN       0.96      0.96      0.96     36238
         NUM       0.86      0.95      0.90      2119
        PART       0.96      0.85      0.90      5117
        PRON       0.92      0.92      0.92      7444
       PROPN       0.87      0.97      0.92      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.84      0.95      0.89      2859
         SYM       0.79      0.98      0.88        62
        VERB     

## Применение полученных теггеров и сравнение

In [71]:
class POSTagger:
    def __init__(self, model, char2id, id2label, max_sent_len, max_token_len):
        self.model = model
        self.char2id = char2id
        self.id2label = id2label
        self.max_sent_len = max_sent_len
        self.max_token_len = max_token_len

    def __call__(self, sentences):
        tokenized_corpus = tokenize_corpus(sentences, min_token_size=1)

        inputs = torch.zeros((len(sentences), self.max_sent_len, self.max_token_len + 2), dtype=torch.long)

        for sent_i, sentence in enumerate(tokenized_corpus):
            for token_i, token in enumerate(sentence):
                for char_i, char in enumerate(token):
                    inputs[sent_i, token_i, char_i + 1] = self.char2id.get(char, 0)

        dataset = TensorDataset(inputs, torch.zeros(len(sentences)))
        predicted_probs = predict_with_model(self.model, dataset)  # SentenceN x TagsN x MaxSentLen
        predicted_classes = predicted_probs.argmax(1)

        result = []
        for sent_i, sent in enumerate(tokenized_corpus):
            result.append([self.id2label[cls] for cls in predicted_classes[sent_i, :len(sent)]])
        return result

single_token_pos_tagger = POSTagger(single_token_model, char_vocab, 
                                    UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, 
                                      UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

In [72]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [73]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 100.77it/s]                    

мама-NOUN мыла-NOUN раму-NOUN

косил-VERB косой-NOUN косой-NOUN косой-NOUN

глокая-VERB куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-ADV три-NUM будет-AUX дырка-NOUN

три-NUM да-ADV три-NUM будет-AUX шесть-NUM

сорок-NOUN сорок-NOUN



In [74]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 104.10it/s]                    

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-NOUN косой-NOUN косой-NOUN

глокая-ADJ куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-PART три-NUM будет-VERB дырка-NOUN

три-NUM да-PART три-NUM будет-VERB шесть-VERB

сорок-NOUN сорок-NOUN



## Свёрточный модуль своими руками

In [ ]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [ ]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

In [ ]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

In [ ]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))